In [ ]:
import numpy as np
import pandas as pd

In [ ]:
players = pd.read_csv('../dataset light memory/players_fifa22.csv')
teams = pd.read_csv('../dataset light memory/teams_fifa22.csv')

In [ ]:
list(players.columns)

In [ ]:
['GK', 
 'LB', 'LCB', 'RCB', 'RB', 
 'CML', 'CAM', 'CMR',
 'LW', 'ST', 'RW']

In [ ]:
players.columns

In [ ]:
import random
from collections import deque
import os
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
        
class DQNetwork:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.01
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim = self.state_size, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        model.compile(loss='mse', optimizer = Adam(learning_rate = self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in batch:
            target = reward #target = 80
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0])) 
            target_cur = self.model.predict(state)
            target_cur[0][action] = target
            self.model.fit(state, target_cur, epochs = 1, verbose = 0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    def load(self, name):
        self.model.load_weights(name)
    def save(self, name):
        self.model.save_weights(name)
    


In [ ]:
import gym

env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1001

In [ ]:
# agent = DQNetwork(state_size, action_size)

# for e in range(n_episodes):
#     state = env.reset()
#     state = np.reshape(state, [1, state_size])
#     for time in range(5000):
#         #env.render()
#         action = agent.act(state)
#         next_state, reward, done, _ = env.step(action)
#         reward = reward if not done else -10
#         next_state = np.reshape(next_state, [1, state_size])
#         agent.remember(state, action, reward, next_state, done)
#         state = next_state
#         if done:
#             print(f"episode: {e}/{n_episodes}, score: {time}, e: {round(agent.epsilon, 2)}")
#             break
#     if len(agent.memory) > batch_size:
#         agent.replay(batch_size)

In [ ]:
state = env.reset()
state = np.reshape(state, [1, state_size])
next_state, reward, done, _ = env.step(action)

In [ ]:
testdqn = DQNetwork(10, 2)

In [ ]:
testdqn.model.predict([[1,2,3,4,5,6,7,8,9,10]])

In [ ]:
class Player:
    def __init__(self, name = '', position = '', position_rating = 0, value = 0, wage = 0):
        self.name = name
        #self.overall_rating = overall_rating
        self.position = position
        self.position_rating = position_rating
        self.value = value
        #self.wage = wage
        
class Environment:
    def __init__(self, players, money):
        self.players = players[['ID', 'Name', 'ValueEUR', 'Overall', 'STRating', 'LWRating', 'LFRating', 'CFRating', 'RFRating', 'RWRating',
                                'CAMRating', 'LMRating', 'CMRating', 'RMRating', 'LWBRating',
                                'CDMRating', 'RWBRating', 'LBRating', 'CBRating', 'RBRating', 'GKRating']]
        self.money = money
        self.formation = ['GK', 
                         'LB', 'CB', 'CB', 'RB', 
                         'LM', 'CAM', 'RM',
                         'LW', 'ST', 'RW']
        self.squad = self.buildSquad()
        
    def buildSquad(self):
        squad = []
        for i in range(11):
            cur_position = self.formation.pop()
            player_id = np.random.randint(0, 1000, 1)[0]
            temp_player = Player(players.iloc[player_id]['Name'], 
                                 cur_position,
                                 players.iloc[player_id][f'{cur_position}Rating'],
                                 players.iloc[player_id]['ValueEUR'])
            squad.append(temp_player)
        return squad
    def evalSquad(self):
        sum_rating = 0
        for i in self.squad:
            sum_rating += i.position_rating
        return round(sum_rating/11, 2)

In [ ]:
env = Environment(players, 1000000)

In [ ]:
for player in env.squad:
    print(f'name: {player.name}, position: {player.position}, Position rating: {player.position_rating}')

In [ ]:
print(env.evalSquad())